# 科学技術計算第5回課題

In [1]:
import numpy as np
import scipy

from numpy.linalg import inv, norm, solve, det, matrix_rank, cond, pinv, lstsq, eig, eigh, eigvals, eigvalsh
from numpy import diag

import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams["savefig.bbox"] = "tight"

from typing import List, Optional, Union, Tuple

import sklearn
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_olivetti_faces

from matplotlib.pyplot import imshow

# 5-1

In [22]:
def householder_step(
    A: np.ndarray,
    k: int
) -> Tuple[np.ndarray, np.ndarray]:
    """k-th step of Householder transform from nxn symmetric to tridiagonal

    Args:
        A (np.ndarray): an nxn symmetric matrix
        k (int): step

    Returns:
        Tuple[np.ndarray, np.ndarray]: transformed matrix QAQ after k-th step, and the transfrom Q
    """
    assert A.ndim == 2
    assert A.shape[0] == A.shape[1]

    n = A.shape[0]
    assert 0 <= k < n - 2

    x = A[k + 1:, k]
    y = np.zeros_like(x)
    y[0] = - np.sign(A[k + 1, k]) * norm(x)

    u = (x - y) / norm(x - y)
    H = np.eye(len(x)) - 2 * np.outer(u, u)

    Q = np.eye(n)
    Q[k + 1:, k + 1:] = H

    QAQ = Q @ A @ Q
    return QAQ, Q

def householder_transform(
    A: np.ndarray,
) -> Tuple[np.ndarray, np.ndarray]:
    """Householder transform from nxn symmetric to tridiagonal

    Args:
        A (np.ndarray): an nxn symmetric matrix

    Returns:
        Tuple[np.ndarray, np.ndarray]: tridiagonal Atri and orthogonal Qhh
        so that Qhh A Qhh == Atri
    """
    assert A.ndim == 2
    assert A.shape[0] == A.shape[1]
    n = A.shape[0]

    Atri = A.copy()
    Qhh = np.eye(n)
    for k in range(n - 2):
        Atri, Q = householder_step(Atri, k)
        Qhh = Qhh @ Q
    return Atri, Qhh

def givens_r_matrix(
    xp_xq: np.ndarray
) -> np.ndarray:
    """generate a matrix of Givens rotation

    Args:
        xp_xq (ndarray): two values xp, xq

    Returns:
        ndarray: 2D rotation matrix
    """
    assert len(xp_xq) == 2

    xp, xq = xp_xq[0], xp_xq[1]

    # denom = np.sqrt(xp**2 + xq**2)
    denom = norm(xp_xq)
    c = xp / denom
    s = -xq / denom

    R = np.array([
        [c, s],
        [-s, c]
    ])
    return R

def qr_decom_by_givens(
    A: np.ndarray,
) -> Tuple[np.ndarray, np.ndarray]:
    """QR decomposition of a tridiagonal matrix by Givens rotation

    Args:
        A (np.ndarray): an nxn tridiagonal matrix

    Returns:
        Tuple[np.ndarray, np.ndarray]: orthogonal Q and upper traiangular R so that A == QR
    """
    assert len(A.shape) == 2
    assert A.shape[0] == A.shape[1]
    # assert A should be tridiagonal
    n = A.shape[0]

    AG = A.copy()
    Q = np.eye(n)

    for k in range(n - 1):
        p, q = k, k + 1
        Rot2x2 = givens_r_matrix(AG[[p, q]][:, p])
        AG[[p, q]] = Rot2x2.T @ AG[[p, q]]
        Q[:, [p, q]] = Q[:, [p, q]] @ Rot2x2

    R = AG
    return Q, R

def rayleigh_quotient_iteration(
    A: np.ndarray,
    mu: float,
    u: np.ndarray,
    maxiter: int = 5000,
) -> Tuple[float, np.ndarray]:
    """Rayleigh quotioent iteration for computing the largest eigenvalue and its eigenvector of A

    Args:
        A (np.ndarray): nxn matrix A
        mu (float): initial eigenvalue
        u (np.ndarray): n-d initial eigenvector
        maxiter (int, optional): max iterations. Defaults to 50.

    Returns:
        Tuple[float, np.ndarray]: the largest eigenvalue and its eigenvector
    """
    assert A.ndim == 2
    assert A.shape[0] == A.shape[1]
    n = A.shape[0]
    assert len(u) == n

    u_pre = u.copy()
    for i in range(maxiter):
        u = solve(A - mu * np.eye(n), u)
        u /= norm(u)
        mu = u.T @ A @ u

        if np.allclose(u, u_pre) or np.allclose(u, -u_pre):
            break
        u_pre = u.copy()

    return mu, u

def my_eigh(A: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    A_tri, Q_hh = householder_transform(A)
    max_iter = 100
    for _ in range(max_iter):
        Q, R = qr_decom_by_givens(A_tri)
        A_next = R @ Q
        if np.allclose(np.diag(A_tri), np.diag(A_next), atol=1e-8):
            break
        A_tri = A_next
    eigenvalues = np.diag(A_tri)
    eigenvectors = []
    for mu in eigenvalues:
        _, eigenvector = rayleigh_quotient_iteration(A, mu, np.ones(A.shape[0]))
        eigenvectors.append(eigenvector)

    eigenvectors = np.array(eigenvectors).T
    eigenvectors /= norm(eigenvectors, axis=0)

    return eigenvalues, Q_hh @ eigenvectors


In [23]:
Atri = np.array([[4, -1, 0], [-1, 4, -1], [0, -1, 3]])
my_eigenvalues, my_eigenvectors = my_eigh(Atri)
print("my_eigenvalues\n", my_eigenvalues)
print("my_eigenvector\n", my_eigenvectors, sep='')
np_eigenvalues, np_eigenvectors = np.linalg.eigh(Atri)
print("np_eigenvalues\n", np_eigenvalues , sep='')
print("np_eigenvector\n", np_eigenvectors , sep='')
print("my_eigenvalues == np_eigenvalues", np.allclose(my_eigenvalues, np_eigenvalues))
print("my_eigenvectors == np_eigenvectors", np.allclose(my_eigenvectors, np_eigenvectors ))

my_eigenvalues
 [5.24693107 3.55500542 2.19806351]
my_eigenvector
[[-0.59100905 -0.73697623  0.32798528]
 [-0.73697623  0.32798528 -0.59100905]
 [-0.32798528  0.59100905  0.73697623]]
np_eigenvalues
[2.19806226 3.55495813 5.2469796 ]
np_eigenvector
[[-0.32798528  0.73697623 -0.59100905]
 [-0.59100905  0.32798528  0.73697623]
 [-0.73697623 -0.59100905 -0.32798528]]
my_eigenvalues == np_eigenvalues False
my_eigenvectors == np_eigenvectors False


# 5-2

# 5-3